In [1]:
# Imports
import numpy as np
import math
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.layouts import column
from sklearn import datasets, metrics

In [2]:
# Data importing and organizing

yacht_file = open("./YachtRegression.txt","r")
data = yacht_file.readlines()

typenames = None
dataset = []
for line in data:
    if(line[0]=="#"):
        typenames = [x[1:].replace("_"," ") for x in line.split()]
    else:
        if(len(dataset)<50):
            words = line.split()
            dataset.append([float(x) for x in words])

main_val = 5
regr_val = 6

X = [[x[main_val]] for x in dataset]
Y = [x[regr_val] for x in dataset]
X = np.asarray(X)
Y = np.asarray(Y)

#X,Y = datasets.make_regression(n_samples=20, n_features=1,n_targets=1,noise=2)

In [52]:
# Alghoritm definition
def lowess(x,X,Y,weights,kernel,h):
    if h == None:
        h = 1
    upper_sum = 0
    down_sum = 0
    for i in range(len(X)):
        upper_sum = upper_sum + Y[i]*weights[i]*kernel(metrics.pairwise.euclidean_distances([x],[X[i]])[0][0]/h)
        down_sum = down_sum + weights[i]*kernel(metrics.pairwise.euclidean_distances([x],[X[i]])[0][0]/h)
    if(upper_sum==0):
        return 0
    return upper_sum/down_sum

def cross_validation(X,Y,kernel,steps,h):
    weights = [1 for x in X]
    errors = [1 for x in X]
    for i in range(steps):
        for k in range(len(X)):
            item = X[k]
            val = Y[k]
            X_t = np.delete(X,k,0)
            Y_t = np.delete(Y,k)
            weights_t = np.delete(weights,k)
            val_t = lowess(item,X_t,Y_t,weights_t,kernel,h)
            errors[k] = abs(val_t-val)
        s = np.median(errors)
        weights = [(1-abs(e/(6*s))**2)**2 if abs((e/(6*s)))<=1 else 0 for e in errors]
    return weights

def kernel_gauss(val):
    return (math.pi**(-0.5))*(math.e**(-(val**2)/2))


def kernel_cvart(val):
    return (15/16)*((1-val**2)**2)*(1 if math.fabs(val)<=1 else 0)

In [68]:
# Calculations
x_min = X[:,:].min()
x_max = X[:,:].max()

h_gaus = 0.025
weights_gauss = np.asarray(cross_validation(X,Y,kernel_gauss,5,h_gaus))
wg_min,wg_max = weights_gauss[:].min(),weights_gauss[:].max()


h_cvart = 0.025
weights_cvart = np.asarray(cross_validation(X,Y,kernel_cvart,5,h_cvart))
wc_min,wc_max = weights_cvart[:].min(),weights_cvart[:].max()


In [69]:
# Visualizations
colors = [[250,0,0],[0,0,250]]
TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,reset,tap,save,"
r = (x_max-x_min)/100

x_steps = 400
x_step = (x_max-x_min)/x_steps


p_1 = figure(tools=TOOLS,x_range=(x_min,x_max))
p_1.scatter([x[0] for x in X], [x for x in Y],radius = r,
          fill_color=["#%02x%02x%02x" % 
                      (int((1-w)*255),int((1-w)*255),int((1-w)*255)) for w in weights_gauss],
          line_color="#000000")
p_1.line([x_min+x_step*x for x in range(x_steps)],
         [lowess([x_min+x_step*x],X,Y,weights_gauss,kernel_gauss,h_gaus) for x in range(x_steps)],
         line_width=2)

p_1.xaxis.axis_label = typenames[main_val]
p_1.yaxis.axis_label = typenames[regr_val]



p_2 = figure(tools=TOOLS,x_range=(x_min,x_max))
p_2.scatter([x[0] for x in X], [x for x in Y],radius = r,
          fill_color=["#%02x%02x%02x" % 
                      (int((1-w)*255),int((1-w)*255),int((1-w)*255)) for w in weights_cvart],
          line_color="#000000")

p_2.line([x_min+x_step*x for x in range(x_steps)],
         [lowess([x_min+x_step*x],X,Y,weights_cvart,kernel_cvart,h_cvart) for x in range(x_steps)],
         line_width=2)

p_2.xaxis.axis_label = typenames[main_val]
p_2.yaxis.axis_label = typenames[regr_val]


output_file("Lowess.html", title="Lowess")
output_notebook()

show(column(p_1,p_2), notebook_handle=True)

Loading BokehJS ...